In [3]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [4]:
# Loading data
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("./AAPL_Mock_ML_Open_v2.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Season_spring,Season_summer,Season_fall,Season_winter,quarter_1,quarter_2,quarter_3,quarter_4,Open_24hr_Change,Gain/Loss_Open
0,1981-01-23,0.146763,0.147321,0.146205,0.146205,0.114768,11222400,0,0,0,1,0,0,0,1,0.146763,Gain
1,1981-01-26,0.144531,0.144531,0.143973,0.143973,0.113016,24640000,0,0,0,1,0,0,0,1,-0.002232,Loss
2,1981-01-27,0.143973,0.143973,0.142857,0.142857,0.112140,23699200,0,0,0,1,0,0,0,1,-0.000558,Loss
3,1981-01-28,0.138951,0.138951,0.138393,0.138393,0.108636,28156800,0,0,0,1,0,0,0,1,-0.005022,Loss
4,1981-01-29,0.133929,0.133929,0.133371,0.133371,0.104694,43904000,0,0,0,1,0,0,0,1,-0.005022,Loss


In [5]:
# Convert date into an integer since the system was not taking it as a float
# https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
apple_df['Date'] = apple_df['Date'].astype(int)

In [6]:
# Define features set
X = apple_df.copy()
X = X.drop("Gain/Loss_Open", axis=1)
X.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Season_spring,Season_summer,Season_fall,Season_winter,quarter_1,quarter_2,quarter_3,quarter_4,Open_24hr_Change
0,349056000000000000,0.146763,0.147321,0.146205,0.146205,0.114768,11222400,0,0,0,1,0,0,0,1,0.146763
1,349315200000000000,0.144531,0.144531,0.143973,0.143973,0.113016,24640000,0,0,0,1,0,0,0,1,-0.002232
2,349401600000000000,0.143973,0.143973,0.142857,0.142857,0.112140,23699200,0,0,0,1,0,0,0,1,-0.000558
3,349488000000000000,0.138951,0.138951,0.138393,0.138393,0.108636,28156800,0,0,0,1,0,0,0,1,-0.005022
4,349574400000000000,0.133929,0.133929,0.133371,0.133371,0.104694,43904000,0,0,0,1,0,0,0,1,-0.005022


In [7]:
# Define the target set.
y = apple_df["Gain/Loss_Open"].ravel()
y[:5]

array(['Gain', 'Loss', 'Loss', 'Loss', 'Loss'], dtype=object)

In [8]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=500, random_state=78) 

In [11]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [12]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [13]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,2,0
Actual 1,0,6


In [14]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [15]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2,0
Actual 1,0,6


Accuracy Score : 1.0
Classification Report
              precision    recall  f1-score   support

        Gain       1.00      1.00      1.00         2
        Loss       1.00      1.00      1.00         6

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [16]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.10539797, 0.06997804, 0.05478702, 0.0534422 , 0.05414468,
       0.04936703, 0.0998058 , 0.01171453, 0.        , 0.        ,
       0.00866323, 0.        , 0.        , 0.        , 0.        ,
       0.4926995 ])

In [17]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.49269949649487077, 'Open_24hr_Change'),
 (0.10539796818259378, 'Date'),
 (0.09980579966332684, 'Volume'),
 (0.0699780436519479, 'Open'),
 (0.05478702292710834, 'High'),
 (0.054144677679982475, 'Close'),
 (0.05344219845806003, 'Low'),
 (0.049367026762653705, 'Adj Close'),
 (0.011714533699551008, 'Season_spring'),
 (0.00866323247990526, 'Season_winter'),
 (0.0, 'quarter_4'),
 (0.0, 'quarter_3'),
 (0.0, 'quarter_2'),
 (0.0, 'quarter_1'),
 (0.0, 'Season_summer'),
 (0.0, 'Season_fall')]